In [1]:
# Updated script: Only list durations for IDs that had a bounding box drawn

import cv2
import time
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# ─── SETTINGS ────────────────────────────────────────────────────────────────
VIDEO_PATH     = "cam_a.mov"
MODEL_PATH     = "yolov8n.pt"
TARGET_FPS     = 25
FRAME_INTERVAL = 1.0 / TARGET_FPS
LINE_Y         = 610
MAX_DIST       = 100     # max vertical distance from LINE_Y to draw boxes
# ──────────────────────────────────────────────────────────────────────────────

# 1) Init detector & tracker
yolo = YOLO(MODEL_PATH)
tracker = DeepSort(
    max_age=40,
    n_init=27,
    max_iou_distance=0.4,
    max_cosine_distance=0.25,
    embedder="torchreid",
    embedder_model_name="osnet_ain_x1_0",
    embedder_gpu=False,
    nn_budget=1200
)

# 2) Open video
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    raise IOError(f"Cannot open video: {VIDEO_PATH}")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))

# 3) State for counting & timing
prev_intersect     = {}   # tid -> was intersecting last frame
total_crosses      = 0    # cumulative crosses
track_start_frame  = {}   # tid -> first frame_idx seen
track_end_frame    = {}   # tid -> frame_idx when disappeared
prev_frame_ids     = set()
drawn_ids          = set()  # track IDs that have had a box drawn

frame_idx = 0
while True:
    t0 = time.perf_counter()
    ret, frame = cap.read()
    if not ret:
        break

    # a) draw the counting line
    #cv2.line(frame, (0, LINE_Y), (width, LINE_Y), (0, 0, 255), 2)

    # b) detect persons
    results = yolo(frame, classes=[0])[0]
    dets = [
        ([int(x1), int(y1), int(x2-x1), int(y2-y1)], float(conf), int(cls))
        for (x1,y1,x2,y2), conf, cls in zip(
            results.boxes.xyxy.cpu().numpy(),
            results.boxes.conf.cpu().numpy(),
            results.boxes.cls.cpu().numpy().astype(int)
        )
    ]

    # c) update tracker
    tracks = tracker.update_tracks(dets, frame=frame)

    # find current confirmed IDs
    current_ids = {tr.track_id for tr in tracks if tr.is_confirmed()}

    # detect disappeared IDs
    disappeared = prev_frame_ids - current_ids
    for tid in disappeared:
        if tid not in track_end_frame:
            track_end_frame[tid] = frame_idx
    prev_frame_ids = current_ids

    # d) count intersections & draw boxes, ID, and live timer
    current_intersect = 0
    for tr in tracks:
        if not tr.is_confirmed():
            continue
        x1, y1, x2, y2 = map(int, tr.to_ltrb())
        tid = tr.track_id

        # compute vertical centroid
        cy = (y1 + y2) // 2
        # skip drawing if too far from line
        if abs(cy - LINE_Y) > MAX_DIST:
            prev_intersect[tid] = False
            continue

        # record start frame
        if tid not in track_start_frame:
            track_start_frame[tid] = frame_idx

        # mark that this ID was drawn
        drawn_ids.add(tid)

        # does this box intersect the line?
        intersects = (y1 <= LINE_Y <= y2)
        if intersects:
            current_intersect += 1
        was = prev_intersect.get(tid, False)
        if not was and intersects:
            total_crosses += 1
        prev_intersect[tid] = intersects

        # draw bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        # draw ID above box
        cv2.putText(frame, f"ID:{tid}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
        # draw live elapsed time below box
        elapsed_secs = (frame_idx - track_start_frame[tid]) / TARGET_FPS
        cv2.putText(frame, f"{elapsed_secs:.1f}s", (x1, y1-25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)

    # e) display live counts
    cv2.putText(frame, f"In Line Now:   {current_intersect}", (10,30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
    #cv2.putText(frame, f"Total Crosses: {total_crosses}", (10,60),
                #cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    # f) print disappeared durations only for drawn IDs
    y = 100
    for tid, end_f in track_end_frame.items():
        if tid not in drawn_ids:
            continue
        start_f = track_start_frame.get(tid, 0)
        duration = (end_f - start_f) / TARGET_FPS
        text = f"ID {tid} lasted {duration:.1f}s"
        cv2.putText(frame, text, (10, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200,200,200), 2)
        y += 25

    # g) show
    cv2.imshow("Live Count with Disappearance Durations", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # h) sync to TARGET_FPS, skipping if needed
    elapsed = time.perf_counter() - t0
    wait    = FRAME_INTERVAL - elapsed
    if wait > 0:
        cv2.waitKey(int(wait * 1000))
    else:
        skips = int(-wait / FRAME_INTERVAL)
        for _ in range(skips):
            cap.grab()
        frame_idx += skips

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()


/Users/fadisultan/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/fadisultan/anaconda3/lib/python3.10/site-packages/torchreid/utils/tools.py:43: UserWarning: No file found at "/Users/fadisultan/anaconda3/lib/python3.10/site-packages/deep_sort_realtime/embedder/weights/osnet_ain_ms_d_c_wtsonly.pth"
  warnings.warn('No file found at "{}"'.format(fpath))


Successfully loaded imagenet pretrained weights from "/Users/fadisultan/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352

0: 384x640 11 persons, 47.4ms
Speed: 1.8ms preprocess, 47.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 45.3ms
Speed: 1.4ms preprocess, 45.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 53.7ms
Speed: 1.4ms preprocess, 53.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 45.6ms
Speed: 1.6ms preprocess, 45.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 persons, 52.5ms
Speed: 2.0ms preprocess, 52.5ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 11 persons, 47.4ms
Speed: 1.8ms preprocess,